<a href="https://colab.research.google.com/github/HuyenNguyenHelen/Huyen_INFO5731_Spring2020/blob/master/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [2019 Dell labtop](https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/dp/B07N49F51N/ref=sr_1_11?crid=1IJ7UWF2F4GHH&keywords=dell%2Bxps%2B15&qid=1580173569&sprefix=dell%2Caps%2C181&sr=8-11&th=1) on amazon.

(2) Collect the top 100 User Reviews of the film [Joker](https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv) from IMDB.

(3) Collect the abstracts of the top 100 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(4) Collect the top 100 tweets by using hashtag ["#wuhancoronovirus"](https://twitter.com/hashtag/wuhancoronovirus) from Twitter. 


In [0]:
# Write your code here
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv"
request = urllib.request.urlopen (url)
webpage  = request.read()
soup = BeautifulSoup(webpage)

review = soup.find('select', {'class':'lister-list' })
#print(review)

content_soup=soup.find_all("div", class_="text show-more__control")
title_soup = soup.find_all("a", class_="title")
date_soup = soup.find_all("span", class_="review-date")
rating_soup = soup.find_all("span", class_="rating-other-user-rating")
users_soup = soup.find_all("span", class_="display-name-link" )

user_names, user_links = [],[]
for tag in users_soup:
  href = tag.find_all("a")
  for h in href:
   user_links.append(h.get("href"))
   user_names. append(h.get_text())

title, content, date, rating = [], [], [], []
for a,b,c,d,e in zip(content_soup,title_soup, date_soup, rating_soup, users_soup):
  content.append(a.get_text())
  title.append(b.get_text())
  date.append(c.get_text())
  rating.append((d.getText()).strip())

dict = {"review_title": title, "review_content": content, "rates": rating,  "reviewer": user_names, "reviewer_profile_link": user_links, "review_date": date}
df = pd.DataFrame(dict)
df
with open("E:\Helen\INFO5731_Assignment2\Joke_reviews.csv", "w", newline="") as file:
  df.to_csv(file)
with open("E:\Helen\INFO5731_Assignment2\Joke_reviews.csv", "r") as file:
  pd.read_csv(file)



In [41]:
#generate links of loading more
def next_page ():
  load_more_data = soup.find("div", class_="load-more-data" ) # find this button at the end of each page
  #print(load_more_data)
  path_parse1 = "https://www.imdb.com"
  path_parse2 = load_more_data.get("data-ajaxurl")
  path_parse3 = "?ref_=undefined&paginationKey="
  path_parse4 = load_more_data.get("data-key")
  concatenated_path = path_parse1+path_parse2+path_parse3+path_parse4
  print(concatenated_path)


https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7crnqq5dgzqh7kwxtnrzrdtmwbbhzfmxvlnomwklyczuf43o6ss6oe3v3nzidb4k4b2nzddokl2hrk4s66zdfw4aidq


In [40]:
from urllib.parse import urlparse
urlparse("https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv")

ParseResult(scheme='https', netloc='www.imdb.com', path='/title/tt7286456/reviews', params='', query='ref_=tt_urv', fragment='')

In [45]:
# Write your code here
# Amazon Reviews_Dell
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/product-reviews/B07N49F51N/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"
request = urllib.request.urlopen (url)
webpage  = request.read()
soup = BeautifulSoup(webpage)




HTTPError: ignored

In [0]:
review = soup.find('select', {'class':'lister-list' })
#print(review)

content_soup=soup.find_all("div", class_="text show-more__control")
title_soup = soup.find_all("a", class_="title")
date_soup = soup.find_all("span", class_="review-date")
rating_soup = soup.find_all("span", class_="rating-other-user-rating")
users_soup = soup.find_all("span", class_="display-name-link" )

user_names, user_links = [],[]
for tag in users_soup:
  href = tag.find_all("a")
  for h in href:
   user_links.append(h.get("href"))
   user_names. append(h.get_text())

title, content, date, rating = [], [], [], []
for a,b,c,d,e in zip(content_soup,title_soup, date_soup, rating_soup, users_soup):
  content.append(a.get_text())
  title.append(b.get_text())
  date.append(c.get_text())
  rating.append((d.getText()).strip())

dict = {"review_title": title, "review_content": content, "rates": rating,  "reviewer": user_names, "reviewer_profile_link": user_links, "review_date": date}
df = pd.DataFrame(dict)
df
with open("E:\Helen\INFO5731_Assignment2\Joke_reviews.csv", "w", newline="") as file:
  df.to_csv(file)
with open("E:\Helen\INFO5731_Assignment2\Joke_reviews.csv", "r") as file:
  pd.read_csv(file)


# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [0]:
# Write your code here





# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [0]:
# Write your code here






**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 